<a href="https://colab.research.google.com/github/SurPaul/stable-diffusion-webui-colab/blob/main/Stable_Diffusion_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Настройка

#@markdown #### 1. Сохранить папку `outputs` на Google Диск? Она будет сохранена в `/Colab Notebooks`.
OUTPUT_DIR_SYNC = True #@param {type: "boolean"}
OUTPUT_DIR_ON_DRIVE = "Colab Notebooks"

#@markdown #### 2. [Получите](https://huggingface.co/settings/tokens) токен на Hugging Face 🤗
HUGGINGFACE_TOKEN = "" #@param {type: "string"}

#@markdown #### 3. Выберите версию Stable Diffusion для скачивания:
SD_VERSION = "v1-5-pruned-emaonly.ckpt (4.27 \u0413\u0431)" #@param ["v1-5-pruned-emaonly.ckpt (4.27 Гб)", "arcane-diffusion-v3.ckpt (2.13 Гб)", "analog-diffusion-1.0.ckpt (2.13 Гб)", "dreamlike-diffusion-1.0.ckpt (2.13 Гб)", "dreamlike-photoreal-1.0.ckpt (2.13 Гб)", "mdjrny-v4.ckpt (2.13 Гб)", "redshift-diffusion-v1.ckpt (2.13 Гб)"]

#@markdown После выбора нужных параметров вы можете нажать `Ctrl + F9` (`Среда выполнения` > `Выполнить всё`), чтобы запустить WebUI.</br>Ссылка на веб-интерфейс (`*.gradio.app`) через несколько минут появится в ячейке `Запуск WebUI`.

sd_models_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
user_header = f"\"Authorization: Bearer {HUGGINGFACE_TOKEN}\""

!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
#@title Скачивание и установка
#@markdown * Подключение Google Диска *(опционально)*;
#@markdown * Установка Conda, Xformers;
#@markdown * Скачивание Stable Diffusion с Hugging Face 🤗
#@markdown * Веб-интерфейс [AUTOMATIC1111/stable-diffusion-webui](https://github.com/AUTOMATIC1111/stable-diffusion-webui);
#@markdown * Расширение [yfszzx/stable-diffusion-webui-images-browser](https://github.com/yfszzx/stable-diffusion-webui-images-browser) для просмотра результатов из веб-интерфейса;
#@markdown * Расширение [yownas/seed_travel](https://github.com/yownas/seed_travel).

#@title Symlink между GDrive и Outputs
from os.path import exists
from os import symlink, getenv, environ
from google.colab import drive, output

if OUTPUT_DIR_SYNC:
  if not exists("/content/drive"):
    drive.mount("/content/drive")

  src = f"/content/drive/MyDrive/{OUTPUT_DIR_ON_DRIVE}/outputs"
  dst = "/content/stable-diffusion-webui/outputs"

  if exists(src) == False:
    %mkdir "{src}"

DB_ENABLE = False
environ["PYTHONPATH"] = ""
environ["CONDA_ALWAYS_YES"] = "true"
if not exists("/usr/local/bin/conda"):
  !wget -cq https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
  !chmod +x Miniconda3-latest-Linux-x86_64.sh
  !./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
  !conda install xformers -c xformers/label/dev
  if DB_ENABLE:
    !conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

!git clone -q https://github.com/AUTOMATIC1111/stable-diffusion-webui
!git clone -q https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone -q https://github.com/yownas/seed_travel /content/stable-diffusion-webui/extensions/seed_travel
!pip install -q moviepy==1.0.3

if OUTPUT_DIR_SYNC:
  if exists(dst) == False:
    symlink(src, dst)

if DB_ENABLE:
  !git clone -q https://github.com/d8ahazard/sd_dreambooth_extension /content/stable-diffusion-webui/extensions/sd_dreambooth_extension
  !pip install -q -r /content/stable-diffusion-webui/extensions/sd_dreambooth_extension/requirements.txt

output.clear()
conf_dir = "/content/stable-diffusion-webui/configs"
conf = ""
ckpt = ""

if not exists(conf_dir):
  %mkdir {conf_dir}
  !wget --header={user_header} -q https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-inference.yaml -O {conf_dir}/v1-inference.yaml
  !wget -q https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml -O {conf_dir}/v2-inference.yaml
  !wget -q https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O {conf_dir}/v2-inference-v.yaml

if SD_VERSION == "v1-5-pruned-emaonly.ckpt (4.27 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O {sd_models_dir}/v1-5-pruned-emaonly.ckpt
  conf = f"--config {conf_dir}/v1-inference.yaml"
  ckpt = f"--ckpt {sd_models_dir}/v1-5-pruned-emaonly.ckpt"

elif SD_VERSION == "v1-5-pruned.ckpt (7.7 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt -O {sd_models_dir}/v1-5-pruned.ckpt
  conf = f"--config {conf_dir}/v1-inference.yaml"
  ckpt = f"--ckpt {sd_models_dir}/v1-5-pruned.ckpt"

elif SD_VERSION == "v2-1_512-ema-pruned.ckpt (5.21 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt -O {sd_models_dir}/v2-1_512-ema-pruned.ckpt
  conf = f"--config {conf_dir}/v2-inference.yaml"
  ckpt = f"--ckpt {sd_models_dir}/v2-1_512-ema-pruned.ckpt"

elif SD_VERSION == "v2-1_768-ema-pruned.ckpt (5.21 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O {sd_models_dir}/v2-1_768-ema-pruned.ckpt
  conf = f"--config {conf_dir}/v2-inference-v.yaml"
  ckpt = f"--ckpt {sd_models_dir}/v2-1_768-ema-pruned.ckpt"

elif SD_VERSION == "arcane-diffusion-v3.ckpt (2.13 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/nitrosocke/Arcane-Diffusion/resolve/main/arcane-diffusion-v3.ckpt -O {sd_models_dir}/arcane-diffusion-v3.ckpt
  conf = f"--config {conf_dir}/v1-inference.yaml"
  ckpt = f"--ckpt {sd_models_dir}/arcane-diffusion-v3.ckpt"

elif SD_VERSION == "analog-diffusion-1.0.ckpt (2.13 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/wavymulder/Analog-Diffusion/resolve/main/analog-diffusion-1.0.ckpt -O {sd_models_dir}/analog-diffusion-1.0.ckpt
  conf = f"--config {conf_dir}/v1-inference.yaml"
  ckpt = f"--ckpt {sd_models_dir}/analog-diffusion-1.0.ckpt"

elif SD_VERSION == "dreamlike-diffusion-1.0.ckpt (2.13 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/dreamlike-art/dreamlike-diffusion-1.0/resolve/main/dreamlike-diffusion-1.0.ckpt -O {sd_models_dir}/dreamlike-diffusion-1.0.ckpt
  conf = f"--config {conf_dir}/v1-inference.yaml"
  ckpt = f"--ckpt {sd_models_dir}/dreamlike-diffusion-1.0.ckpt"

elif SD_VERSION == "dreamlike-photoreal-1.0.ckpt (2.13 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/dreamlike-art/dreamlike-photoreal-1.0/resolve/main/dreamlike-photoreal-1.0.ckpt -O {sd_models_dir}/dreamlike-photoreal-1.0.ckpt
  conf = f"--config {conf_dir}/v1-inference.yaml"
  ckpt = f"--ckpt {sd_models_dir}/dreamlike-photoreal-1.0.ckpt"

elif SD_VERSION == "mdjrny-v4.ckpt (2.13 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt -O {sd_models_dir}/mdjrny-v4.ckpt
  conf = f"--config {conf_dir}/v1-inference.yaml"
  ckpt = f"--ckpt {sd_models_dir}/mdjrny-v4.ckpt"

elif SD_VERSION == "redshift-diffusion-v1.ckpt (2.13 Гб)":
  !wget --header={user_header} -cq --show-progress https://huggingface.co/nitrosocke/redshift-diffusion/resolve/main/redshift-diffusion-v1.ckpt -O {sd_models_dir}/redshift-diffusion-v1.ckpt
  conf = f"--config {conf_dir}/v1-inference.yaml"
  ckpt = f"--ckpt {sd_models_dir}/redshift-diffusion-v1.ckpt"

output.clear()

In [ ]:
#@title Запуск WebUI
%cd /content/stable-diffusion-webui
!python launch.py --share --enable-insecure-extension-access {ckpt} {conf}